# Model Predictive Control
The general idea of figuring out what moves to make using optimisation at each time step has become very popular due to the fact that a general version can be programmed and made very user friendly so that the intricacies of multivariable control can be handled by a single program.

In this notebook I will show how a single time step's move trajectory is calculated. We'll use the same system as we used for the [Dahlin controller](../6_Discrete_control_and_analysis/Dahlin%20controller.ipynb)

We start with a linear model of the system

$$G=\frac{1}{15s^2 + 8s + 1}$$

#### Press ▶️ to plot the step response of a defined linear system.

In [1]:
import numpy
import scipy.signal
import scipy.optimize
import matplotlib.pyplot as plt
%matplotlib inline

G = scipy.signal.lti([1], [15, 8, 1])
plt.plot(*G.step())

print('Operation Complete')

ModuleNotFoundError: No module named 'numpy'

Our goal is to find out what manipulations must be made (changes to $u$) in order to get the system to follow a specific desired trajectory (which we will call $r$ for the reference trajectory). We will allow the controller to make a certain number of moves. This is called the control horizon, $M$. We will the observe the effect of this set of moves (called a "move plan") for time called the prediction horizon ($P$).

### Controller parameters
#### Press ▶️ to define control and prediction horizons, set the sampling rate, and generate time points for continuous and discrete representations over the prediction horizon.

In [ ]:
M = 10  # Control horizon
P = 20  # Prediction horizon
DeltaT = 1  # Sampling rate

tcontinuous = numpy.linspace(0, P*DeltaT, 1000)  # some closely spaced time points
tpredict = numpy.arange(0, P*DeltaT, DeltaT)   # discrete points at prediction horizon
print('Operation Complete')

### We choose a first order setpoint response similar to DS or Dahlin
#### Press ▶️ to calculate the response curve based on a time constant and the exponential decay formula.

In [ ]:
tau_c = 1
r = 1 - numpy.exp(-tpredict/tau_c)
print('Operation Complete')

### For an initial guess we choose a step in $u$.
#### Press ▶️ to create an array of control inputs initialized to one for the specified control horizon.

In [ ]:
u = numpy.ones(M)
x0 = numpy.zeros(G.to_ss().A.shape[0])
print('Operation Complete')

#### Press ▶️ to extend the control input array, simulate the system response, and plot the predicted output over the prediction horizon.

In [ ]:
def extend(u):
    """We optimise the first M values of u but we need P values for prediction"""
    return numpy.concatenate([u, numpy.repeat(u[-1], P-M)])

def prediction(u, t=tpredict, x0=x0):
    """Predict the effect of an input signal"""
    t, y, x = scipy.signal.lsim(G, u, t, X0=x0, interp=False)
    return y

plt.plot(tpredict, prediction(extend(u)))
print('Operation Complete')

#### Press ▶️ to define a function that computes the objective value by evaluating the squared error between the predicted and desired responses.

In [ ]:
def objective(u, x0=x0):
    """Calculate the sum of the square error for the cotnrol problem"""
    y = prediction(extend(u))
    return sum((r - y)**2)
print('Operation Complete')

#### Press ▶️ to obtain the value of the objective for our step input:

In [ ]:
objective(u)

#### Press ▶️ to get a set of moves which will minimise our objective function

In [ ]:
result = scipy.optimize.minimize(objective, u)
uopt = result.x
result.fun

#### Press ▶️ to resample the discrete output to continuous time, effectively working out the zero-order hold value.

In [ ]:
ucont = extend(uopt)[((tcontinuous-0.01)//DeltaT).astype(int)]
print('Operation Complete')

#### Press ▶️ to plot the move plan and the output, verifying that the output aligns with the desired values at the sampling times, effectively recovering the Dahlin controller.

In [ ]:
def plotoutput(ucont, uopt):
    plt.figure()
    plt.plot(tcontinuous, ucont)
    plt.xlim([0, DeltaT*(P+1)])
    plt.figure()
    plt.plot(tcontinuous, prediction(ucont, tcontinuous), label='Continuous response')
    plt.plot(tpredict, prediction(extend(uopt)), '-o', label='Optimized response')
    plt.plot(tpredict, r, label='Set point')
    plt.legend()

plotoutput(ucont, uopt)
print('Operation Complete')

One of the reasons for the popularity of MPC is how easy it is to change its behaviour using weights in the objective function. Try using this definition instead of the simple one above and see if you can remove the ringing in the controller output.

#### Press ▶️ to modify the behavior of the MPC by adjusting weights in the objective function, aiming to eliminate ringing in the controller output.

In [ ]:
def objective(u, x0=x0):
    y = prediction(extend(u))
    umag = numpy.abs(u)
    constraintpenalty = sum(umag[umag > 2])
    movepenalty = sum(numpy.abs(numpy.diff(u)))
    strongfinish = numpy.abs(y[-1] - r[-1])
    return sum((r - y)**2) + 0*constraintpenalty + 0.1*movepenalty + 0*strongfinish

objective(u, x0=x0)